In [11]:
from sympy import *

In [64]:
# Define variables
I = Symbol('I', real=True)
R = IndexedBase('R', real=True)
beta = Symbol("beta", real=True, positive=True)
rho = Symbol("rho", real=True, positive=True)
T = Symbol('T', real=True, positive=True)
m = symbols('m', cls=Idx)

def generate_flow(N: int):
    flow = [beta * I * (1 - I - Sum(R[m], (m, 1, N))).doit() - rho * I, # I'
            rho * I - N/T * R[1]]                                       # R_1'
    for mm in range(2, N+1):
        flow += [(R[mm-1] - R[mm]) * N/T]                               # R_m'
    return Matrix(flow)

def var_vec(N: int):
    return Matrix([I, *(R[mm] for mm in range(1, N+1))])

# Fixpoints

In [83]:
def flow_to_eq(flow: Matrix):
    return [Eq(f, 0) for f in flow]

def find_fixpoints(N: int):
    sys = flow_to_eq(generate_flow(N))
    return solve(sys, var_vec(N), dict=True)


N = 5
display(find_fixpoints(N))

[{I: 0, R[1]: 0, R[2]: 0, R[3]: 0, R[4]: 0, R[5]: 0},
 {I: (beta - rho)/(beta*(T*rho + 1)),
  R[1]: T*rho*(beta - rho)/(5*beta*(T*rho + 1)),
  R[2]: T*rho*(beta - rho)/(5*beta*(T*rho + 1)),
  R[3]: T*rho*(beta - rho)/(5*beta*(T*rho + 1)),
  R[4]: T*rho*(beta - rho)/(5*beta*(T*rho + 1)),
  R[5]: T*rho*(beta - rho)/(5*beta*(T*rho + 1))}]

# Jacobian

In [84]:
N = 3
J = generate_flow(N).jacobian(var_vec(N))
simplify(J.subs(find_fixpoints(N)[1]))

Matrix([
[(-beta + rho)/(T*rho + 1), (-beta + rho)/(T*rho + 1), (-beta + rho)/(T*rho + 1), (-beta + rho)/(T*rho + 1)],
[                      rho,                      -3/T,                         0,                         0],
[                        0,                       3/T,                      -3/T,                         0],
[                        0,                         0,                       3/T,                      -3/T]])